## GOLD & S&P500 post 1 month and 2 month after rate cuts

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

def get_yf_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close']

rate_cut_dates = [
    '2001-01-03', '2001-04-18', '2001-09-17', '2001-11-06',
    '2008-01-22', '2008-10-08', '2020-03-03', '2020-03-15'
]

start_date = datetime.strptime(min(rate_cut_dates), '%Y-%m-%d') - timedelta(days=30)
end_date = datetime.strptime(max(rate_cut_dates), '%Y-%m-%d') + timedelta(days=60)

gold = get_yf_data('GC=F', start_date, end_date)
sp500 = get_yf_data('^GSPC', start_date, end_date)

data = pd.concat([gold, sp500], axis=1)
data.columns = ['Gold', 'S&P 500']

def calculate_change(data, start_date, months):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = start + timedelta(days=30*months)
    window = data.loc[start:end]
    if len(window) > 1:
        return ((window.iloc[-1] / window.iloc[0] - 1) * 100).fillna(0)
    return pd.Series([0, 0], index=data.columns)

changes_1m = pd.DataFrame([calculate_change(data, date, 1) for date in rate_cut_dates], index=rate_cut_dates)
changes_2m = pd.DataFrame([calculate_change(data, date, 2) for date in rate_cut_dates], index=rate_cut_dates)

plt.style.use('dark_background')

def create_plot(changes, title):
    fig, ax = plt.subplots(figsize=(16, 10))
    fig.patch.set_facecolor('#0E1117')
    ax.set_facecolor('#0E1117')

    bar_width = 0.35
    index = np.arange(len(rate_cut_dates))
    colors = ['#FFD700', '#4169E1']
    assets = ['Gold', 'S&P 500']

    for j, column in enumerate(assets):
        valid_mask = np.isfinite(changes[column])
        bars = ax.bar(index[valid_mask] + j*bar_width, changes[column][valid_mask], 
                      bar_width, label=column, color=colors[j], alpha=0.8)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}%', ha='center', va='bottom', fontsize=10, color='white')

    ax.set_xlabel('Rate Cut Dates', fontsize=14, labelpad=15)
    ax.set_ylabel('Percentage Change (%)', fontsize=14, labelpad=15)
    ax.set_title(title, fontsize=20, fontweight='bold', pad=20)

    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([datetime.strptime(date, '%Y-%m-%d').strftime('%b %d, %Y') for date in rate_cut_dates], rotation=45, ha='right')
    ax.tick_params(axis='both', which='major', labelsize=12)

    ax.legend(fontsize=12, loc='upper left', bbox_to_anchor=(1, 1))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='y', which='both', left=False)
    ax.grid(axis='y', linestyle='--', alpha=0.2)
    ax.axhline(y=0, color='white', linestyle='-', linewidth=0.5, alpha=0.5)

    cut_magnitudes = {'2001-01-03': '50bps', '2001-04-18': '50bps', '2001-09-17': '50bps', 
                      '2001-11-06': '50bps', '2008-01-22': '75bps', '2008-10-08': '50bps', 
                      '2020-03-03': '50bps', '2020-03-15': '100bps'}

    for k, date in enumerate(rate_cut_dates):
        ax.text(k, ax.get_ylim()[0], cut_magnitudes[date], ha='center', va='bottom', fontsize=10, color='white', alpha=0.7, rotation=45)

    fig.text(0.02, 0.02, 'Data source: Yahoo Finance (Gold, S&P 500)\n'
             'Bars show percentage change after rate cut. Rate cut magnitudes shown below dates.', 
             fontsize=10, color='#CCCCCC', alpha=0.7)

    plt.tight_layout()
    return fig

fig1 = create_plot(changes_1m, 'Gold and S&P 500 Reaction 1 Month After Fed Rate Cuts')
fig2 = create_plot(changes_2m, 'Gold and S&P 500 Reaction 2 Months After Fed Rate Cuts')

fig1.savefig('gold_sp500_reaction_1month_after_rate_cuts.png', dpi=300, bbox_inches='tight', facecolor='#0E1117')
fig2.savefig('gold_sp500_reaction_2months_after_rate_cuts.png', dpi=300, bbox_inches='tight', facecolor='#0E1117')

print("Visualizations saved as 'gold_sp500_reaction_1month_after_rate_cuts.png' and 'gold_sp500_reaction_2months_after_rate_cuts.png'")